In [ ]:
# 필수 라이브러리 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score, f1_score
from tqdm import tqdm
import time, os, json, warnings
import joblib
import pickle
from datetime import datetime
import gc

# 선택적 라이브러리 import
try:
    import xgboost as xgb
    HAS_XGB = True
except ImportError:
    HAS_XGB = False

try:
    import lightgbm as lgb
    HAS_LGB = True
except ImportError:
    HAS_LGB = False

try:
    import catboost as cb
    HAS_CATBOOST = True
except ImportError:
    HAS_CATBOOST = False

try:
    import shap
    HAS_SHAP = True
except ImportError:
    HAS_SHAP = False

# 한글 폰트 설정
warnings.filterwarnings("ignore")
try:
    plt.rcParams['font.family'] = 'AppleGothic' # 맥
except Exception:
    plt.rcParams['font.family'] ='Malgun Gothic' # 윈도우
plt.rcParams['axes.unicode_minus'] = False

print("="*80)
print("하수처리장 예측 파이프라인 초기화 완료")
print(f"XGBoost: {'✓' if HAS_XGB else '✗'}")
print(f"LightGBM: {'✓' if HAS_LGB else '✗'}")
print(f"CatBoost: {'✓' if HAS_CATBOOST else '✗'}")
print(f"SHAP: {'✓' if HAS_SHAP else '✗'}")
print("="*80)

# 각 SHAP 분석 후 메모리 정리
import gc
plt.close('all')  # 모든 matplotlib 그래프 닫기
gc.collect()      # 가비지 컬렉션 강제 실행

In [ ]:
'''
# ================================================================================================
# 완전한 하수처리장 예측 파이프라인 (학습-저장-예측)
# ================================================================================================

# ================================================================================================
# 1. 모델 정의 함수들
# ================================================================================================
def build_regression_models():
    """회귀 모델들"""
    models = {}
    models["RandomForest_Reg"] = RandomForestRegressor(
        n_estimators=300, min_samples_leaf=2, random_state=42, n_jobs=-1
    )
    models["LinearRegression"] = LinearRegression()
    models["GradientBoosting_Reg"] = GradientBoostingRegressor(
        n_estimators=200, learning_rate=0.1, random_state=42
    )
    if HAS_XGB:
        models["XGBoost_Reg"] = xgb.XGBRegressor(
            n_estimators=400, max_depth=5, learning_rate=0.05,
            subsample=0.8, colsample_bytree=0.8,
            random_state=42, n_jobs=-1, verbosity=0
        )
    if HAS_LGB:
        models["LightGBM_Reg"] = lgb.LGBMRegressor(
            n_estimators=500, learning_rate=0.05,
            subsample=0.8, colsample_bytree=0.8,
            random_state=42, n_jobs=-1, verbosity=-1
        )
    if HAS_CATBOOST:
        models["CatBoost_Reg"] = cb.CatBoostRegressor(
            iterations=500, learning_rate=0.05, depth=6,
            random_state=42, verbose=False
        )
    return models

def build_classification_models():
    """분류 모델들 (4등급)"""
    models = {}
    models["RandomForest_Clf"] = RandomForestClassifier(
        n_estimators=300, min_samples_leaf=2, random_state=42, 
        n_jobs=-1, class_weight='balanced'
    ) 
    models["GradientBoosting_Clf"] = GradientBoostingClassifier(
        n_estimators=200, learning_rate=0.1, random_state=42
    )
    models["LogisticRegression_Clf"] = LogisticRegression(
        multi_class="multinomial", solver="lbfgs", max_iter=1000,
        random_state=42, class_weight='balanced'
    )
    if HAS_XGB:
        models["XGBoost_Clf"] = xgb.XGBClassifier(
            n_estimators=400, max_depth=5, learning_rate=0.05,
            subsample=0.8, colsample_bytree=0.8,
            objective="multi:softprob", num_class=4,
            tree_method="hist", random_state=42, n_jobs=-1, verbosity=0
        )
    if HAS_LGB:
        models["LightGBM_Clf"] = lgb.LGBMClassifier(
            n_estimators=500, learning_rate=0.05,
            subsample=0.8, colsample_bytree=0.8,
            objective="multiclass", num_class=4,
            random_state=42, n_jobs=-1, verbosity=-1, is_unbalance=True
        )
    if HAS_CATBOOST:
        models["CatBoost_Clf"] = cb.CatBoostClassifier(
            iterations=500, learning_rate=0.05, depth=6,
            random_state=42, verbose=False, auto_class_weights='Balanced'
        )
    return models

# ================================================================================================
# 2. 데이터 처리 함수들
# ================================================================================================
def make_pipeline_unified(model, model_name, model_type):
    """통합 전처리 파이프라인"""
    if model_name in ["LinearRegression", "LogisticRegression_Clf"]:
        pre = Pipeline(steps=[
            ("imputer", SimpleImputer(strategy="median")),
            ("scaler", StandardScaler()),
        ])
    else:
        pre = Pipeline(steps=[
            ("imputer", SimpleImputer(strategy="median")),
        ])
    return Pipeline(steps=[("pre", pre), ("model", model)])

def prepare_data_stratified(df, target_col, model_type, test_size=0.2, split_method='stratified'):
    """데이터 준비 - Stratified vs 시계열 분할"""
    work = df.sort_values('날짜').reset_index(drop=True).copy()
    dates = pd.to_datetime(work['날짜'])

    not_use_col = [
        '날짜',
        '1처리장','2처리장','정화조','중계펌프장','합계','시설현대화',
        '3처리장','4처리장','합계', '합계_1일후','합계_2일후',
        '등급','등급_1일후','등급_2일후'
    ]
    
    drop_cols = [c for c in (set(not_use_col) | {target_col}) if c in work.columns]
    X_raw = work.drop(columns=drop_cols, errors="ignore")
    
    for c in X_raw.columns:
        X_raw[c] = pd.to_numeric(X_raw[c], errors="coerce")

    if model_type == "regression":
        y = pd.to_numeric(work[target_col], errors="coerce")
    else:
        y = work[target_col].astype("int64")

    valid_idx = (~X_raw.isnull().all(axis=1)) & (~pd.isnull(y))
    X_raw = X_raw[valid_idx].reset_index(drop=True)
    y = y[valid_idx].reset_index(drop=True)
    dates = dates[valid_idx].reset_index(drop=True)
    
    if split_method == 'stratified':
        if model_type == "classification":
            sss = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=42)
            train_idx, test_idx = next(sss.split(X_raw, y))
        else:
            train_idx, test_idx = train_test_split(
                range(len(X_raw)), test_size=test_size, random_state=42
            )
            
        X_train, X_test = X_raw.iloc[train_idx].copy(), X_raw.iloc[test_idx].copy()
        y_train, y_test = y.iloc[train_idx].copy(), y.iloc[test_idx].copy()
        dates_train, dates_test = dates.iloc[train_idx].copy(), dates.iloc[test_idx].copy()
        
    else:  # temporal split
        n = len(X_raw)
        split = int(n * (1 - test_size))
        X_train, X_test = X_raw.iloc[:split].copy(), X_raw.iloc[split:].copy()
        y_train, y_test = y.iloc[:split].copy(), y.iloc[split:].copy()
        dates_train, dates_test = dates.iloc[:split].copy(), dates.iloc[split:].copy()

    feature_names = list(X_raw.columns)
    return X_train, X_test, y_train, y_test, feature_names, dates_train, dates_test

def make_features(df, cutoff_date=None):
    """파생변수 생성 함수 - Data Leakage 방지 버전"""
    df = df.copy()
    
    df['날짜'] = pd.to_datetime(df['날짜'])
    df = df.sort_values('날짜').reset_index(drop=True)
    
    # 디버깅 정보 - 시작 시점
    print(f"\n=== make_features 디버깅 ===")
    print(f"입력 데이터: {len(df)}행")
    print(f"날짜 범위: {df['날짜'].min()} ~ {df['날짜'].max()}")
    if cutoff_date is not None:
        cutoff = pd.to_datetime(cutoff_date)
        print(f"cutoff_date: {cutoff_date}")
        print(f"  cutoff 이전 데이터: {len(df[df['날짜'] <= cutoff])}행")
        print(f"  cutoff 이후 데이터: {len(df[df['날짜'] > cutoff])}행")

    df['월'] = df['날짜'].dt.month
    df['요일'] = df['날짜'].dt.weekday

    season_map = {'봄': 0, '여름': 1, '가을': 2, '겨울': 3}
    discomfort_map = {'쾌적': 0, '약간 불쾌': 1, '불쾌': 2, '매우 불쾌': 3, '극심한 불쾌': 4}
    df['계절'] = df['계절'].map(season_map).astype('Int64')
    df['불쾌지수등급'] = df['불쾌지수등급'].map(discomfort_map).astype('Int64')

    # 강수량 시차 피처
    df['강수량_1일전'] = df['일_일강수량(mm)'].shift(1)
    df['강수량_2일전'] = df['일_일강수량(mm)'].shift(2)
    df['강수량_1일_누적'] = df['일_일강수량(mm)'].rolling(1, min_periods=1).sum()
    df['강수량_2일_누적'] = df['일_일강수량(mm)'].rolling(2, min_periods=1).sum()
    df['강수량_3일_누적'] = df['일_일강수량(mm)'].rolling(3, min_periods=1).sum()
    df['강수량_5일_누적'] = df['일_일강수량(mm)'].rolling(5, min_periods=1).sum()
    df['강수량_7일_누적'] = df['일_일강수량(mm)'].rolling(7, min_periods=1).sum()

    df['일교차'] = df['일_최고기온(°C)'] - df['일_최저기온(°C)']
    df['폭우_여부'] = (df['일_일강수량(mm)'] >= 80).astype(int)
    
    # 체감온도 계산 (간단 버전)
    T = pd.to_numeric(df.get('일_평균기온(°C)', np.nan), errors='coerce')
    V_ms = pd.to_numeric(df.get('일_평균풍속(m/s)', np.nan), errors='coerce')
    RH = pd.to_numeric(df.get('평균습도(%)', np.nan), errors='coerce')
    
    e = (RH/100.0) * 6.105 * np.exp(17.27*T/(237.7 + T))
    df['체감온도(°C)'] = T + 0.33*e - 0.70*V_ms - 4.00
    
    # 분류용 등급 계산
    q = df['합계'].dropna().quantile([0.15, 0.70, 0.90])
    q15, q70, q90 = float(q.loc[0.15]), float(q.loc[0.70]), float(q.loc[0.90])
    print(f"등급 구분 기준: q15={q15:.0f}, q70={q70:.0f}, q90={q90:.0f}")

    def categorize(x):
        if pd.isna(x):
            return np.nan
        if x < q15:
            return 0
        elif x < q70:
            return 1
        elif x < q90:
            return 2
        else:
            return 3

    df['등급'] = df['합계'].apply(categorize)
    
    # 타겟 변수 생성 (Data Leakage 방지)
    if cutoff_date is not None:
        cutoff = pd.to_datetime(cutoff_date)
        print(f"타겟 변수 생성 (cutoff 적용)")
        
        df['합계_1일후'] = np.nan
        df['합계_2일후'] = np.nan
        df['등급_1일후'] = np.nan
        df['등급_2일후'] = np.nan
        
        # 생성된 타겟 개수 추적
        target_1day_count = 0
        target_2day_count = 0
        
        for i in range(len(df)):
            current_date = df.loc[i, '날짜']
            
            if i + 1 < len(df) and current_date <= cutoff:
                next_date = df.loc[i+1, '날짜']
                if next_date <= cutoff:
                    df.loc[i, '합계_1일후'] = df.loc[i+1, '합계']
                    df.loc[i, '등급_1일후'] = df.loc[i+1, '등급']
                    target_1day_count += 1
            
            if i + 2 < len(df) and current_date <= cutoff:
                next2_date = df.loc[i+2, '날짜']
                if next2_date <= cutoff:
                    df.loc[i, '합계_2일후'] = df.loc[i+2, '합계']
                    df.loc[i, '등급_2일후'] = df.loc[i+2, '등급']
                    target_2day_count += 1
                    
        print(f"  1일후 타겟 생성: {target_1day_count}개")
        print(f"  2일후 타겟 생성: {target_2day_count}개")
        
    else:
        print(f"타겟 변수 생성 (shift 사용)")
        df['합계_1일후'] = df['합계'].shift(-1)
        df['합계_2일후'] = df['합계'].shift(-2)
        df['등급_1일후'] = df['등급'].shift(-1).astype('Int64')
        df['등급_2일후'] = df['등급'].shift(-2).astype('Int64')
        print(f"  1일후 타겟 유효값: {df['합계_1일후'].notna().sum()}개")
        print(f"  2일후 타겟 유효값: {df['합계_2일후'].notna().sum()}개")

    df.attrs['cutoffs'] = {"q15": q15, "q70": q70, "q90": q90}
    
    # dropna 전후 비교
    before_dropna = len(df)
    df = df.dropna().reset_index(drop=True)
    after_dropna = len(df)
    print(f"dropna 처리: {before_dropna}행 → {after_dropna}행 ({before_dropna - after_dropna}행 제거)")
    
    # 2025-06-01 필터링
    before_filter = len(df)
    df = df[df["날짜"] < "2025-06-01"]
    after_filter = len(df)
    print(f"날짜 필터링: {before_filter}행 → {after_filter}행 ({before_filter - after_filter}행 제거)")
    
    # 최종 결과
    if '등급_1일후' in df.columns:
        final_grade_dist = df['등급_1일후'].value_counts().sort_index()
        print(f"최종 등급 분포: {dict(final_grade_dist)}")
    
    print(f"최종 출력: {len(df)}행, {len(df.columns)}컬럼")
    print(f"최종 날짜 범위: {df['날짜'].min()} ~ {df['날짜'].max()}")
    print(f"=== make_features 완료 ===\n")
    
    return df

def make_features_for_prediction(historical_df, future_df):
    """새로운 데이터에 대한 파생변수 생성 (과거 데이터 활용)"""
    combined_df = pd.concat([historical_df, future_df], ignore_index=True)
    combined_df['날짜'] = pd.to_datetime(combined_df['날짜'])
    combined_df = combined_df.sort_values('날짜').reset_index(drop=True)
    
    combined_df['월'] = combined_df['날짜'].dt.month
    combined_df['요일'] = combined_df['날짜'].dt.weekday
    
    season_map = {'봄': 0, '여름': 1, '가을': 2, '겨울': 3}
    discomfort_map = {'쾌적': 0, '약간 불쾌': 1, '불쾌': 2, '매우 불쾌': 3, '극심한 불쾌': 4}
    combined_df['계절'] = combined_df['계절'].map(season_map).astype('Int64')
    combined_df['불쾌지수등급'] = combined_df['불쾌지수등급'].map(discomfort_map).astype('Int64')
    
    # 시차 변수들
    combined_df['강수량_1일전'] = combined_df['일_일강수량(mm)'].shift(1)
    combined_df['강수량_2일전'] = combined_df['일_일강수량(mm)'].shift(2)
    combined_df['강수량_1일_누적'] = combined_df['일_일강수량(mm)'].rolling(1, min_periods=1).sum()
    combined_df['강수량_2일_누적'] = combined_df['일_일강수량(mm)'].rolling(2, min_periods=1).sum()
    combined_df['강수량_3일_누적'] = combined_df['일_일강수량(mm)'].rolling(3, min_periods=1).sum()
    combined_df['강수량_5일_누적'] = combined_df['일_일강수량(mm)'].rolling(5, min_periods=1).sum()
    combined_df['강수량_7일_누적'] = combined_df['일_일강수량(mm)'].rolling(7, min_periods=1).sum()
    
    combined_df['일교차'] = combined_df['일_최고기온(°C)'] - combined_df['일_최저기온(°C)']
    combined_df['폭우_여부'] = (combined_df['일_일강수량(mm)'] >= 80).astype(int)
    
    # 체감온도 계산
    T = pd.to_numeric(combined_df.get('일_평균기온(°C)', np.nan), errors='coerce')
    V_ms = pd.to_numeric(combined_df.get('일_평균풍속(m/s)', np.nan), errors='coerce')
    RH = pd.to_numeric(combined_df.get('평균습도(%)', np.nan), errors='coerce')
    
    e = (RH/100.0) * 6.105 * np.exp(17.27*T/(237.7 + T))
    combined_df['체감온도(°C)'] = T + 0.33*e - 0.70*V_ms - 4.00
    
    # 새 데이터 부분만 반환
    historical_len = len(historical_df)
    return combined_df.iloc[historical_len:].reset_index(drop=True)

def make_simple_features(data):
    """간단한 피처 생성 (시차 변수 제외)"""
    df = data.copy()
    df['날짜'] = pd.to_datetime(df['날짜'])
    df = df.sort_values('날짜').reset_index(drop=True)
    
    # 기본 피처들
    df['월'] = df['날짜'].dt.month
    df['요일'] = df['날짜'].dt.weekday
    
    # 계절/불쾌지수 매핑
    season_map = {'봄': 0, '여름': 1, '가을': 2, '겨울': 3}
    discomfort_map = {'쾌적': 0, '약간 불쾌': 1, '불쾌': 2, '매우 불쾌': 3, '극심한 불쾌': 4}
    
    if '계절' in df.columns:
        df['계절'] = df['계절'].map(season_map).astype('Int64')
    if '불쾌지수등급' in df.columns:
        df['불쾌지수등급'] = df['불쾌지수등급'].map(discomfort_map).astype('Int64')
    
    # 기본 계산 피처들
    if '일_최고기온(°C)' in df.columns and '일_최저기온(°C)' in df.columns:
        df['일교차'] = df['일_최고기온(°C)'] - df['일_최저기온(°C)']
    
    if '일_일강수량(mm)' in df.columns:
        df['폭우_여부'] = (df['일_일강수량(mm)'] >= 80).astype(int)
    
    # 체감온도 계산
    T = pd.to_numeric(df.get('일_평균기온(°C)', np.nan), errors='coerce')
    V_ms = pd.to_numeric(df.get('일_평균풍속(m/s)', np.nan), errors='coerce')
    RH = pd.to_numeric(df.get('평균습도(%)', np.nan), errors='coerce')
    
    e = (RH/100.0) * 6.105 * np.exp(17.27*T/(237.7 + T))
    df['체감온도(°C)'] = T + 0.33*e - 0.70*V_ms - 4.00
    
    return df

# ================================================================================================
# 3. 평가 함수들
# ================================================================================================
def evaluate_regression_model(model, model_name, X_train, X_test, y_train, y_test):
    """회귀 모델 평가"""
    try:
        pipe = make_pipeline_unified(model, model_name, "regression")
        pipe.fit(X_train, y_train)
        
        y_pred = pipe.predict(X_test)
        
        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)
        mape = np.mean(np.abs((y_test - y_pred) / (y_test + 1e-8))) * 100
        
        return {
            'model': model_name,
            'type': 'regression',
            'mae': mae,
            'rmse': rmse,
            'r2': r2,
            'mape': mape,
            'success': True
        }, pipe, y_pred
        
    except Exception as e:
        return {
            'model': model_name,
            'type': 'regression',
            'mae': np.nan,
            'rmse': np.nan,
            'r2': np.nan,
            'mape': np.nan,
            'success': False,
            'error': str(e)
        }, None, None

def evaluate_classification_model(model, model_name, X_train, X_test, y_train, y_test):
    """분류 모델 평가"""
    try:
        pipe = make_pipeline_unified(model, model_name, "classification")
        pipe.fit(X_train, y_train)
        
        y_pred = pipe.predict(X_test)
        
        if isinstance(y_pred, np.ndarray) and y_pred.ndim > 1:
            y_pred = y_pred.ravel()
        
        acc = accuracy_score(y_test, y_pred)
        f1_macro = f1_score(y_test, y_pred, average="macro", zero_division=0)
        f1_weighted = f1_score(y_test, y_pred, average="weighted", zero_division=0)
        
        extreme_classes = [0, 3]
        y_true_extreme = pd.Series(y_test).isin(extreme_classes).astype(int)
        y_pred_extreme = pd.Series(y_pred).isin(extreme_classes).astype(int)
        extreme_f1 = f1_score(y_true_extreme, y_pred_extreme, zero_division=0)
        
        return {
            'model': model_name,
            'type': 'classification',
            'accuracy': acc,
            'macro_f1': f1_macro,
            'weighted_f1': f1_weighted,
            'extreme_f1': extreme_f1,
            'success': True
        }, pipe, y_pred
        
    except Exception as e:
        return {
            'model': model_name,
            'type': 'classification',
            'accuracy': np.nan,
            'macro_f1': np.nan,
            'weighted_f1': np.nan,
            'extreme_f1': np.nan,
            'success': False,
            'error': str(e)
        }, None, None

def comprehensive_evaluation_comparison(center_name, df):
    """Stratified vs 시계열 분할 비교 평가"""
    print(f"\n{'='*70}")
    print(f"센터: {center_name} - Stratified vs 시계열 분할 비교")
    print(f"{'='*70}")
    
    print(f"데이터 크기: {len(df)}행, {len(df.columns)}컬럼")
    
    if '등급_1일후' in df.columns:
        grade_dist = df['등급_1일후'].value_counts().sort_index()
        print(f"등급 분포: {dict(grade_dist)}")
        
        min_class = grade_dist.min()
        max_class = grade_dist.max()
        imbalance_ratio = max_class / min_class
        print(f"클래스 불균형 비율: {imbalance_ratio:.1f}:1 (최대:{max_class}, 최소:{min_class})")
    
    results = []
    
    for split_method in ['temporal', 'stratified']:
        print(f"\n{'='*50}")
        print(f"분할 방법: {split_method.upper()}")
        print(f"{'='*50}")
        
        # 회귀 모델 평가
        reg_method_name = "random_shuffle" if split_method == "stratified" else split_method
        print(f"\n--- 회귀 모델 평가 ({reg_method_name}) ---")
        
        try:
            X_train, X_test, y_train, y_test, feature_names, dates_train, dates_test = prepare_data_stratified(
                df, target_col="합계_1일후", model_type="regression", test_size=0.2, split_method=split_method
            )
            
            print(f"회귀용 데이터: 학습 {len(X_train)}행, 테스트 {len(X_test)}행")
            
            regression_models = build_regression_models()
            
            for model_name, model in tqdm(regression_models.items(), desc=f"회귀({reg_method_name})", leave=False):
                result, pipe, y_pred = evaluate_regression_model(model, model_name, X_train, X_test, y_train, y_test)
                result['center'] = center_name
                result['split_method'] = split_method
                results.append(result)
                
                if result['success']:
                    print(f"  {model_name:18s}: R²={result['r2']:.3f}, MAE={result['mae']:.0f}, MAPE={result['mape']:.1f}%")
                else:
                    print(f"  {model_name:18s}: 실패 - {result.get('error', '')[:50]}")
                    
        except Exception as e:
            print(f"회귀 모델 평가 실패 ({reg_method_name}): {e}")
        
        # 분류 모델 평가
        print(f"\n--- 분류 모델 평가 ({split_method}) ---")
        
        try:
            X_train_clf, X_test_clf, y_train_clf, y_test_clf, feature_names_clf, _, _ = prepare_data_stratified(
                df, target_col="등급_1일후", model_type="classification", test_size=0.2, split_method=split_method
            )
            
            print(f"분류용 데이터: 학습 {len(X_train_clf)}행, 테스트 {len(X_test_clf)}행")
            
            test_dist = pd.Series(y_test_clf).value_counts().sort_index()
            train_dist = pd.Series(y_train_clf).value_counts().sort_index()
            print(f"학습 세트 등급 분포: {dict(train_dist)}")
            print(f"테스트 세트 등급 분포: {dict(test_dist)}")
            
            classification_models = build_classification_models()
            
            for model_name, model in tqdm(classification_models.items(), desc=f"분류({split_method})", leave=False):
                result, pipe, y_pred = evaluate_classification_model(model, model_name, X_train_clf, X_test_clf, y_train_clf, y_test_clf)
                result['center'] = center_name
                result['split_method'] = split_method
                results.append(result)
                
                if result['success']:
                    print(f"  {model_name:18s}: ACC={result['accuracy']:.3f}, F1={result['macro_f1']:.3f}, 극값F1={result['extreme_f1']:.3f}")
                else:
                    print(f"  {model_name:18s}: 실패 - {result.get('error', '')[:50]}")
                    
        except Exception as e:
            print(f"분류 모델 평가 실패 ({split_method}): {e}")
    
    return results

# ================================================================================================
# 4. Feature Importance & SHAP 분석 함수들 (선택사항)
# ================================================================================================
def extract_feature_importance(model, model_name, feature_names):
    """모델별 Feature Importance 추출"""
    try:
        mdl = model.named_steps['model']
        if hasattr(mdl, 'feature_importances_'):
            importance = mdl.feature_importances_
        elif hasattr(mdl, 'coef_'):
            coef = mdl.coef_
            if isinstance(coef, np.ndarray) and coef.ndim == 2:
                importance = np.mean(np.abs(coef), axis=0)
            else:
                importance = np.abs(coef)
        else:
            return None

        if len(importance) != len(feature_names):
            print(f"[경고] importance 길이({len(importance)}) != feature_names({len(feature_names)})")
            m = min(len(importance), len(feature_names))
            importance = np.asarray(importance)[:m]
            feature_names = list(feature_names)[:m]

        importance_df = pd.DataFrame({
            'feature': feature_names,
            'importance': importance
        }).sort_values('importance', ascending=False)

        return importance_df
    except Exception as e:
        print(f"Feature importance 추출 실패 ({model_name}): {e}")
        return None

def plot_feature_importance(importance_df, model_name, top_n=15):
    """Feature Importance 시각화"""
    if importance_df is None or len(importance_df) == 0:
        return None
    
    fig, ax = plt.subplots(figsize=(10, 8))
    top_features = importance_df.head(top_n)
    
    ax.barh(range(len(top_features)), top_features['importance'], color='skyblue')
    ax.set_yticks(range(len(top_features)))
    ax.set_yticklabels(top_features['feature'])
    ax.set_xlabel('Importance')
    ax.set_title(f'{model_name} - Top {top_n} Feature Importance')
    ax.invert_yaxis()
    
    for i, v in enumerate(top_features['importance']):
        ax.text(v + 0.001, i, f'{v:.3f}', va='center')
    
    fig.tight_layout()
    return fig

def analyze_model_with_shap(model, X_test, feature_names, model_name, max_samples=100):
    """SHAP 분석 - 다중분류 대응 강화"""
    if not HAS_SHAP:
        print("SHAP 라이브러리가 설치되지 않았습니다.")
        return None
    
    try:
        if len(X_test) > max_samples:
            sample_idx = np.random.choice(len(X_test), max_samples, replace=False)
            X_sample = X_test.iloc[sample_idx]
        else:
            X_sample = X_test
        
        X_processed = model.named_steps['pre'].transform(X_sample)
        
        if 'RandomForest' in model_name or 'GradientBoosting' in model_name:
            explainer = shap.TreeExplainer(model.named_steps['model'])
        elif 'XGBoost' in model_name:
            explainer = shap.TreeExplainer(model.named_steps['model'])
        elif 'LightGBM' in model_name:
            explainer = shap.TreeExplainer(model.named_steps['model'])
        elif 'CatBoost' in model_name:
            explainer = shap.TreeExplainer(model.named_steps['model'])
        else:
            explainer = shap.LinearExplainer(model.named_steps['model'], X_processed)
        
        shap_values = explainer.shap_values(X_processed)
        
        # 원본 SHAP 값 디버깅
        print(f"원본 SHAP 값 타입: {type(shap_values)}")
        if isinstance(shap_values, list):
            print(f"리스트 길이: {len(shap_values)}")
            if len(shap_values) > 0:
                print(f"첫 번째 요소 shape: {shap_values[0].shape if hasattr(shap_values[0], 'shape') else 'no shape'}")
        elif hasattr(shap_values, 'shape'):
            print(f"배열 shape: {shap_values.shape}")
        
        return shap_values, X_processed, explainer
        
    except Exception as e:
        print(f"SHAP 분석 실패 ({model_name}): {e}")
        return None

def plot_shap_summary(shap_values, X_processed, feature_names, model_name):
    """SHAP Summary Plot - 완전 수정 버전"""
    if shap_values is None or not HAS_SHAP:
        return []

    figs = []
    try:
        # 다중분류 SHAP 값 처리 - 더 세밀한 처리
        shap_values_use = None
        
        if isinstance(shap_values, list):
            print(f"SHAP 값이 리스트, 길이: {len(shap_values)}")
            if len(shap_values) > 0:
                # 다중분류의 경우 첫 번째 클래스만 사용
                shap_values_use = shap_values[0]
                print(f"첫 번째 클래스 shape: {shap_values_use.shape}")
        elif isinstance(shap_values, np.ndarray):
            if shap_values.ndim == 3:
                print(f"3차원 배열: {shap_values.shape}")
                # (samples, features, classes) -> (samples, features)
                shap_values_use = shap_values[:, :, 0]
                print(f"첫 번째 클래스 추출 후: {shap_values_use.shape}")
            elif shap_values.ndim == 2:
                print(f"2차원 배열: {shap_values.shape}")
                shap_values_use = shap_values
            else:
                print(f"예상치 못한 차원: {shap_values.ndim}")
                shap_values_use = shap_values
        else:
            print(f"알 수 없는 타입: {type(shap_values)}")
            shap_values_use = shap_values
        
        # 최종 확인
        if shap_values_use is None:
            print("SHAP 값 처리 실패")
            return []
        
        print(f"최종 사용할 SHAP 값 shape: {shap_values_use.shape}")
        print(f"X_processed shape: {X_processed.shape}")
        print(f"feature_names 길이: {len(feature_names)}")
        
        # 차원 일치성 확인
        if hasattr(shap_values_use, 'shape') and len(shap_values_use.shape) >= 2:
            if shap_values_use.shape[1] != len(feature_names):
                print(f"경고: SHAP 피처 수({shap_values_use.shape[1]}) != feature_names 수({len(feature_names)})")
                # 최소 길이로 맞춤
                min_features = min(shap_values_use.shape[1], len(feature_names))
                shap_values_use = shap_values_use[:, :min_features]
                feature_names = feature_names[:min_features]
                print(f"조정 후 - SHAP: {shap_values_use.shape}, features: {len(feature_names)}")
        
        # Bar plot 시도
        try:
            plt.figure(figsize=(10, 8))
            shap.summary_plot(shap_values_use, X_processed[:len(shap_values_use)],
                              feature_names=feature_names,
                              plot_type="bar", show=False)
            plt.title(f'{model_name} - SHAP Feature Importance')
            figs.append(plt.gcf())
            print("    SHAP bar plot 성공")
        except Exception as e:
            print(f"    SHAP bar plot 실패: {e}")
        
        # Beeswarm plot 시도 (더 까다로움)
        try:
            plt.figure(figsize=(10, 8))
            shap.summary_plot(shap_values_use, X_processed[:len(shap_values_use)],
                              feature_names=feature_names,
                              show=False)
            plt.title(f'{model_name} - SHAP Summary Plot')
            figs.append(plt.gcf())
            print("    SHAP beeswarm plot 성공")
        except Exception as e:
            print(f"    SHAP beeswarm plot 실패: {e}")

    except Exception as e:
        print(f"SHAP 시각화 전체 실패 ({model_name}): {e}")

    plt.close('all')
    return figs

# ================================================================================================
# 5. 유틸리티 함수들
# ================================================================================================
def load_original_data():
    """원본 데이터 로드"""
    nanji_raw = pd.read_csv('../data/processed/center_season/nanji/난지_merged.csv', encoding='utf-8-sig')
    jungnang_raw = pd.read_csv('../data/processed/center_season/jungnang/중랑_merged.csv', encoding='utf-8-sig')
    seonam_raw = pd.read_csv('../data/processed/center_season/seonam/서남_merged.csv', encoding='utf-8-sig')
    tancheon_raw = pd.read_csv('../data/processed/center_season/tancheon/탄천_merged.csv', encoding='utf-8-sig')
    
    return {
        "nanji": nanji_raw,
        "jungnang": jungnang_raw,
        "seonam": seonam_raw,
        "tancheon": tancheon_raw
    }

def prepare_prediction_features(future_data, expected_features):
    """예측용 피처 준비"""
    not_use_col = [
        '날짜', '1처리장','2처리장','정화조','중계펌프장','합계','시설현대화',
        '3처리장','4처리장','합계', '합계_1일후','합계_2일후',
        '등급','등급_1일후','등급_2일후'
    ]
    
    available_cols = [col for col in future_data.columns if col not in not_use_col]
    X_future = future_data[available_cols].copy()
    
    for c in X_future.columns:
        X_future[c] = pd.to_numeric(X_future[c], errors="coerce")
    
    missing_features = set(expected_features) - set(X_future.columns)
    if missing_features:
        for feature in missing_features:
            X_future[feature] = 0
    
    X_future = X_future[expected_features].copy()
    return X_future

# ================================================================================================
# 6. 모델 성능 비교 시각화 함수들
# ================================================================================================
def plot_data_characteristics_comparison(centers_data, results_dir, timestamp):
    """센터별 데이터 특성 비교"""
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('센터별 데이터 특성 비교', fontsize=16, fontweight='bold')
    
    center_stats = []
    
    for center_name, df in centers_data.items():
        df['날짜'] = pd.to_datetime(df['날짜'])
        df = df.sort_values('날짜').reset_index(drop=True)
        
        stats = {
            'center': center_name,
            'total_records': len(df),
            'date_range': (df['날짜'].max() - df['날짜'].min()).days,
            'avg_flow': df['합계'].mean() if '합계' in df.columns else 0,
            'std_flow': df['합계'].std() if '합계' in df.columns else 0,
            'missing_ratio': df.isnull().sum().sum() / (len(df) * len(df.columns))
        }
        center_stats.append(stats)
    
    center_stats_df = pd.DataFrame(center_stats)
    
    # 1. 데이터 양 비교
    axes[0,0].bar(center_stats_df['center'], center_stats_df['total_records'], color='skyblue')
    axes[0,0].set_title('센터별 데이터 양')
    axes[0,0].set_ylabel('레코드 수')
    axes[0,0].tick_params(axis='x', rotation=45)
    for i, v in enumerate(center_stats_df['total_records']):
        axes[0,0].text(i, v + 10, str(v), ha='center', va='bottom')
    
    # 2. 평균 유량 비교
    axes[0,1].bar(center_stats_df['center'], center_stats_df['avg_flow'], color='lightgreen')
    axes[0,1].set_title('센터별 평균 유량')
    axes[0,1].set_ylabel('평균 유량')
    axes[0,1].tick_params(axis='x', rotation=45)
    for i, v in enumerate(center_stats_df['avg_flow']):
        axes[0,1].text(i, v + max(center_stats_df['avg_flow'])*0.01, f'{v:.0f}', ha='center', va='bottom')
    
    # 3. 유량 변동성 비교
    axes[1,0].bar(center_stats_df['center'], center_stats_df['std_flow'], color='coral')
    axes[1,0].set_title('센터별 유량 변동성 (표준편차)')
    axes[1,0].set_ylabel('표준편차')
    axes[1,0].tick_params(axis='x', rotation=45)
    for i, v in enumerate(center_stats_df['std_flow']):
        axes[1,0].text(i, v + max(center_stats_df['std_flow'])*0.01, f'{v:.0f}', ha='center', va='bottom')
    
    # 4. 데이터 완성도
    missing_pct = center_stats_df['missing_ratio'] * 100
    axes[1,1].bar(center_stats_df['center'], 100 - missing_pct, color='gold')
    axes[1,1].set_title('센터별 데이터 완성도')
    axes[1,1].set_ylabel('완성도 (%)')
    axes[1,1].set_ylim(0, 100)
    axes[1,1].tick_params(axis='x', rotation=45)
    for i, v in enumerate(100 - missing_pct):
        axes[1,1].text(i, v + 1, f'{v:.1f}%', ha='center', va='bottom')
    
    plt.tight_layout()
    save_path = os.path.join(results_dir, f"data_characteristics_comparison_{timestamp}.png")
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"  데이터 특성 비교 저장: {os.path.basename(save_path)}")

def plot_model_complexity_vs_performance(training_summary, results_dir, timestamp):
    """모델 복잡도 vs 성능 관계"""
    complexity_map = {
        'LinearRegression': 1, 'LogisticRegression_Clf': 1,
        'RandomForest_Reg': 3, 'RandomForest_Clf': 3,
        'GradientBoosting_Reg': 4, 'GradientBoosting_Clf': 4,
        'XGBoost_Reg': 4, 'XGBoost_Clf': 4,
        'LightGBM_Reg': 4, 'LightGBM_Clf': 4,
        'CatBoost_Reg': 5, 'CatBoost_Clf': 5
    }
    
    if isinstance(training_summary, list):
        summary_df = pd.DataFrame(training_summary)
    else:
        summary_df = training_summary.copy()
    
    summary_df['complexity'] = summary_df['model_name'].map(complexity_map)
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    fig.suptitle('모델 복잡도 vs 성능 관계', fontsize=16, fontweight='bold')
    
    # 회귀 모델
    reg_data = summary_df[summary_df['task_type'] == 'regression'].copy()
    if len(reg_data) > 0:
        reg_data['r2'] = reg_data['performance'].apply(lambda x: x.get('r2', 0))
        
        scatter = axes[0].scatter(reg_data['complexity'], reg_data['r2'], 
                                 c=reg_data.index, cmap='viridis', s=100, alpha=0.7)
        
        for i, row in reg_data.iterrows():
            axes[0].annotate(f"{row['center'][:3]}", 
                           (row['complexity'], row['r2']),
                           xytext=(5, 5), textcoords='offset points', fontsize=8)
        
        axes[0].set_xlabel('모델 복잡도 (1=단순, 5=복잡)')
        axes[0].set_ylabel('R² Score')
        axes[0].set_title('회귀: 복잡도 vs 성능')
        axes[0].grid(True, alpha=0.3)
        axes[0].set_xticks(range(1, 6))
    
    # 분류 모델
    clf_data = summary_df[summary_df['task_type'] == 'classification'].copy()
    if len(clf_data) > 0:
        clf_data['macro_f1'] = clf_data['performance'].apply(lambda x: x.get('macro_f1', 0))
        
        scatter = axes[1].scatter(clf_data['complexity'], clf_data['macro_f1'], 
                                 c=clf_data.index, cmap='plasma', s=100, alpha=0.7)
        
        for i, row in clf_data.iterrows():
            axes[1].annotate(f"{row['center'][:3]}", 
                           (row['complexity'], row['macro_f1']),
                           xytext=(5, 5), textcoords='offset points', fontsize=8)
        
        axes[1].set_xlabel('모델 복잡도 (1=단순, 5=복잡)')
        axes[1].set_ylabel('Macro F1 Score')
        axes[1].set_title('분류: 복잡도 vs 성능')
        axes[1].grid(True, alpha=0.3)
        axes[1].set_xticks(range(1, 6))
    
    plt.tight_layout()
    save_path = os.path.join(results_dir, f"model_complexity_vs_performance_{timestamp}.png")
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"  복잡도 vs 성능 저장: {os.path.basename(save_path)}")

def plot_prediction_difficulty_analysis(training_summary, results_dir, timestamp):
    """센터별 예측 난이도 분석"""
    if isinstance(training_summary, list):
        summary_df = pd.DataFrame(training_summary)
    else:
        summary_df = training_summary.copy()
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('센터별 예측 난이도 분석', fontsize=16, fontweight='bold')
    
    centers = summary_df['center'].unique()
    
    # 1. 센터별 최고 R² 성능
    reg_best_scores = []
    for center in centers:
        center_reg = summary_df[(summary_df['center'] == center) & 
                               (summary_df['task_type'] == 'regression')]
        if len(center_reg) > 0:
            center_reg = center_reg.copy()
            center_reg['r2'] = center_reg['performance'].apply(lambda x: x.get('r2', 0))
            reg_best_scores.append(center_reg['r2'].max())
        else:
            reg_best_scores.append(0)
    
    bars1 = axes[0,0].bar(centers, reg_best_scores, color='skyblue')
    axes[0,0].set_title('센터별 최고 R² 성능 (높을수록 예측 용이)')
    axes[0,0].set_ylabel('최고 R² Score')
    axes[0,0].set_ylim(0, 1)
    axes[0,0].tick_params(axis='x', rotation=45)
    
    for bar, score in zip(bars1, reg_best_scores):
        axes[0,0].text(bar.get_x() + bar.get_width()/2, score + 0.01, 
                      f'{score:.3f}', ha='center', va='bottom')
    
    # 2. 센터별 성능 분산 (일관성)
    reg_score_vars = []
    for center in centers:
        center_reg = summary_df[(summary_df['center'] == center) & 
                               (summary_df['task_type'] == 'regression')]
        if len(center_reg) > 0:
            center_reg = center_reg.copy()
            center_reg['r2'] = center_reg['performance'].apply(lambda x: x.get('r2', 0))
            reg_score_vars.append(center_reg['r2'].std())
        else:
            reg_score_vars.append(0)
    
    bars2 = axes[0,1].bar(centers, reg_score_vars, color='lightcoral')
    axes[0,1].set_title('센터별 성능 분산 (낮을수록 일관성 높음)')
    axes[0,1].set_ylabel('R² Score 표준편차')
    axes[0,1].tick_params(axis='x', rotation=45)
    
    for bar, var in zip(bars2, reg_score_vars):
        axes[0,1].text(bar.get_x() + bar.get_width()/2, var + max(reg_score_vars)*0.01, 
                      f'{var:.3f}', ha='center', va='bottom')
    
    # 3. 센터별 최고 F1 성능
    clf_best_scores = []
    for center in centers:
        center_clf = summary_df[(summary_df['center'] == center) & 
                               (summary_df['task_type'] == 'classification')]
        if len(center_clf) > 0:
            center_clf = center_clf.copy()
            center_clf['macro_f1'] = center_clf['performance'].apply(lambda x: x.get('macro_f1', 0))
            clf_best_scores.append(center_clf['macro_f1'].max())
        else:
            clf_best_scores.append(0)
    
    bars3 = axes[1,0].bar(centers, clf_best_scores, color='lightgreen')
    axes[1,0].set_title('센터별 최고 F1 성능 (높을수록 분류 용이)')
    axes[1,0].set_ylabel('최고 Macro F1 Score')
    axes[1,0].set_ylim(0, 1)
    axes[1,0].tick_params(axis='x', rotation=45)
    
    for bar, score in zip(bars3, clf_best_scores):
        axes[1,0].text(bar.get_x() + bar.get_width()/2, score + 0.01, 
                      f'{score:.3f}', ha='center', va='bottom')
    
    # 4. 종합 예측 난이도 점수
    difficulty_scores = [(r2 + f1) / 2 for r2, f1 in zip(reg_best_scores, clf_best_scores)]
    colors = plt.cm.RdYlGn([score for score in difficulty_scores])
    
    bars4 = axes[1,1].bar(centers, difficulty_scores, color=colors)
    axes[1,1].set_title('종합 예측 용이도 점수')
    axes[1,1].set_ylabel('종합 점수 (R² + F1) / 2')
    axes[1,1].set_ylim(0, 1)
    axes[1,1].tick_params(axis='x', rotation=45)
    
    for bar, score in zip(bars4, difficulty_scores):
        axes[1,1].text(bar.get_x() + bar.get_width()/2, score + 0.01, 
                      f'{score:.3f}', ha='center', va='bottom')
    
    plt.tight_layout()
    save_path = os.path.join(results_dir, f"prediction_difficulty_analysis_{timestamp}.png")
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"  예측 난이도 분석 저장: {os.path.basename(save_path)}")

def plot_split_method_detailed_comparison_fixed(all_training_results, results_dir, timestamp):
    """Split Method 상세 비교 분석 - 수정된 버전"""
    if isinstance(all_training_results, list):
        summary_df = pd.DataFrame(all_training_results)
    else:
        summary_df = all_training_results.copy()
    
    successful_results = summary_df[summary_df['success'] == True].copy()
    
    if len(successful_results) == 0 or 'split_method' not in successful_results.columns:
        print("Split method 정보가 없거나 성공한 결과가 없습니다.")
        return
    
    fig, axes = plt.subplots(2, 3, figsize=(20, 12))
    fig.suptitle('Stratified vs Temporal Split 상세 비교', fontsize=16, fontweight='bold')
    
    # 회귀 데이터 처리
    reg_data = successful_results[successful_results['type'] == 'regression'].copy()
    if len(reg_data) > 0:
        reg_pivot = reg_data.pivot_table(index='center', columns='split_method', values='r2')
        reg_pivot.plot(kind='bar', ax=axes[0,0])
        axes[0,0].set_title('센터별 R² 성능: Split Method 비교')
        axes[0,0].set_ylabel('R² Score')
        axes[0,0].tick_params(axis='x', rotation=45)
        axes[0,0].legend(title='Split Method')
        
        split_methods = reg_data['split_method'].unique()
        r2_by_split = [reg_data[reg_data['split_method'] == method]['r2'].values 
                       for method in split_methods]
        
        axes[0,1].boxplot(r2_by_split, labels=split_methods)
        axes[0,1].set_title('Split Method별 R² 분포')
        axes[0,1].set_ylabel('R² Score')
        axes[0,1].grid(True, alpha=0.3)
        
        if len(split_methods) == 2:
            method1, method2 = split_methods
            perf_diff = []
            centers = reg_data['center'].unique()
            
            for center in centers:
                center_data = reg_data[reg_data['center'] == center]
                perf1 = center_data[center_data['split_method'] == method1]['r2']
                perf2 = center_data[center_data['split
                                                
    return created_plots

# ================================================================================================
# 7. SHAP 분석 및 모델 저장 함수들
# ================================================================================================
# 
# 
def train_and_save_single_model_with_shap(center_name, train_data, best_result, task_type, save_dir, results_dir):
    """개별 모델 학습 및 저장 + SHAP 분석 포함"""
    try:
        model_name = best_result['model']
        split_method = best_result['split_method']
        target_col = "합계_1일후" if task_type == "regression" else "등급_1일후"
        
        X_train, X_test, y_train, y_test, feature_names, _, _ = prepare_data_stratified(
            train_data, target_col=target_col, model_type=task_type, 
            test_size=0.2, split_method=split_method
        )
        
        X_all = pd.concat([X_train, X_test], ignore_index=True)
        y_all = pd.concat([y_train, y_test], ignore_index=True)
        
        if task_type == "regression":
            models = build_regression_models()
        else:
            models = build_classification_models()
        
        model = models[model_name]
        pipeline = make_pipeline_unified(model, model_name, task_type)
        pipeline.fit(X_all, y_all)
        
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"{center_name}_{model_name}_{task_type}_{timestamp}.pkl"
        model_path = os.path.join(save_dir, filename)
        
        model_data = {
            'pipeline': pipeline,
            'feature_names': feature_names,
            'model_name': model_name,
            'center_name': center_name,
            'task_type': task_type,
            'performance': dict(best_result),
            'split_method': split_method,
            'training_date': datetime.now().isoformat(),
            'target_column': target_col
        }
        
        with open(model_path, 'wb') as f:
            pickle.dump(model_data, f)
        
        print(f"    저장됨: {filename}")
        
        analysis_pipeline = make_pipeline_unified(model, model_name, task_type)
        analysis_pipeline.fit(X_train, y_train)
        
        # Feature Importance 분석
        try:
            importance_df = extract_feature_importance(analysis_pipeline, model_name, feature_names)
            if importance_df is not None:
                print(f"    Top 5 피처: {', '.join(importance_df.head(5)['feature'].tolist())}")
                
                fig = plot_feature_importance(importance_df, f"{center_name}_{model_name}")
                if fig:
                    img_path = os.path.join(results_dir, f"feature_importance_{center_name}_{model_name}_{task_type}_{timestamp}.png")
                    fig.savefig(img_path, dpi=300, bbox_inches='tight')
                    plt.close(fig)
                    print(f"    Feature Importance 저장: {os.path.basename(img_path)}")
        except Exception as e:
            print(f"    Feature Importance 분석 실패: {e}")
        
        # SHAP 분석
        if HAS_SHAP:
            try:
                print(f"    SHAP 분석 시작...")
                shap_result = analyze_model_with_shap(analysis_pipeline, X_test, feature_names, model_name, max_samples=50)
                
                if shap_result:
                    shap_values, X_processed, explainer = shap_result
                    shap_figs = plot_shap_summary(shap_values, X_processed, feature_names, f"{center_name}_{model_name}")
                    
                    for i, fig in enumerate(shap_figs):
                        suffix = "bar" if i == 0 else "beeswarm"
                        shap_img_path = os.path.join(results_dir, f"shap_{suffix}_{center_name}_{model_name}_{task_type}_{timestamp}.png")
                        fig.savefig(shap_img_path, dpi=300, bbox_inches='tight')
                        plt.close(fig)
                        print(f"    SHAP {suffix} 저장: {os.path.basename(shap_img_path)}")
                    
                    if isinstance(shap_values, np.ndarray):
                        mean_abs_shap = np.mean(np.abs(shap_values), axis=0)
                        shap_summary_df = pd.DataFrame({
                            'feature': feature_names,
                            'mean_abs_shap': mean_abs_shap
                        }).sort_values('mean_abs_shap', ascending=False)
                        
                        shap_csv_path = os.path.join(results_dir, f"shap_summary_{center_name}_{model_name}_{task_type}_{timestamp}.csv")
                        shap_summary_df.to_csv(shap_csv_path, index=False, encoding='utf-8-sig')
                        print(f"    SHAP 요약 저장: {os.path.basename(shap_csv_path)}")
                
                plt.close('all')
                del shap_values, X_processed
                gc.collect()
                print(f"    메모리 정리 완료")
            except Exception as e:
                print(f"    SHAP 분석 실패: {e}")
        else:
            print(f"    SHAP 라이브러리 없음 - SHAP 분석 건너뜀")
        
        return {
            'model_name': model_name,
            'performance': dict(best_result),
            'saved_path': model_path,
            'feature_names': feature_names,
            'target_column': target_col
        }
        
    except Exception as e:
        print(f"    모델 저장 실패: {e}")
        return None

def select_and_save_best_models_with_shap(center_name, train_data, training_results, save_dir, results_dir):
    """센터별 최고 성능 모델 선택 및 저장 + SHAP 분석"""
    results_df = pd.DataFrame(training_results)
    
    
'''

In [ ]:

# ================================================================================================
# 완전한 하수처리장 예측 파이프라인 (학습-저장-예측)
# ================================================================================================

# ================================================================================================
# 1. 모델 정의 함수들
# ================================================================================================
def build_regression_models():
    """회귀 모델들"""
    models = {}
    models["RandomForest_Reg"] = RandomForestRegressor(
        n_estimators=300, min_samples_leaf=2, random_state=42, n_jobs=-1
    )
    models["LinearRegression"] = LinearRegression()
    models["GradientBoosting_Reg"] = GradientBoostingRegressor(
        n_estimators=200, learning_rate=0.1, random_state=42
    )
    if HAS_XGB:
        models["XGBoost_Reg"] = xgb.XGBRegressor(
            n_estimators=400, max_depth=5, learning_rate=0.05,
            subsample=0.8, colsample_bytree=0.8,
            random_state=42, n_jobs=-1, verbosity=0
        )
    if HAS_LGB:
        models["LightGBM_Reg"] = lgb.LGBMRegressor(
            n_estimators=500, learning_rate=0.05,
            subsample=0.8, colsample_bytree=0.8,
            random_state=42, n_jobs=-1, verbosity=-1
        )
    if HAS_CATBOOST:
        models["CatBoost_Reg"] = cb.CatBoostRegressor(
            iterations=500, learning_rate=0.05, depth=6,
            random_state=42, verbose=False
        )
    return models

def build_classification_models():
    """분류 모델들 (4등급)"""
    models = {}
    models["RandomForest_Clf"] = RandomForestClassifier(
        n_estimators=300, min_samples_leaf=2, random_state=42, 
        n_jobs=-1, class_weight='balanced'
    ) 
    models["GradientBoosting_Clf"] = GradientBoostingClassifier(
        n_estimators=200, learning_rate=0.1, random_state=42
    )
    models["LogisticRegression_Clf"] = LogisticRegression(
        multi_class="multinomial", solver="lbfgs", max_iter=1000,
        random_state=42, class_weight='balanced'
    )
    if HAS_XGB:
        models["XGBoost_Clf"] = xgb.XGBClassifier(
            n_estimators=400, max_depth=5, learning_rate=0.05,
            subsample=0.8, colsample_bytree=0.8,
            objective="multi:softprob", num_class=4,
            tree_method="hist", random_state=42, n_jobs=-1, verbosity=0
        )
    if HAS_LGB:
        models["LightGBM_Clf"] = lgb.LGBMClassifier(
            n_estimators=500, learning_rate=0.05,
            subsample=0.8, colsample_bytree=0.8,
            objective="multiclass", num_class=4,
            random_state=42, n_jobs=-1, verbosity=-1, is_unbalance=True
        )
    if HAS_CATBOOST:
        models["CatBoost_Clf"] = cb.CatBoostClassifier(
            iterations=500, learning_rate=0.05, depth=6,
            random_state=42, verbose=False, auto_class_weights='Balanced'
        )
    return models

# ================================================================================================
# 2. 데이터 처리 함수들
# ================================================================================================
def make_pipeline_unified(model, model_name, model_type):
    """통합 전처리 파이프라인"""
    if model_name in ["LinearRegression", "LogisticRegression_Clf"]:
        pre = Pipeline(steps=[
            ("imputer", SimpleImputer(strategy="median")),
            ("scaler", StandardScaler()),
        ])
    else:
        pre = Pipeline(steps=[
            ("imputer", SimpleImputer(strategy="median")),
        ])
    return Pipeline(steps=[("pre", pre), ("model", model)])

def prepare_data_stratified(df, target_col, model_type, test_size=0.2, split_method='stratified'):
    """데이터 준비 - Stratified vs 시계열 분할"""
    work = df.sort_values('날짜').reset_index(drop=True).copy()
    dates = pd.to_datetime(work['날짜'])

    not_use_col = [
        '날짜',
        '1처리장','2처리장','정화조','중계펌프장','합계','시설현대화',
        '3처리장','4처리장','합계', '합계_1일후','합계_2일후',
        '등급','등급_1일후','등급_2일후'
    ]
    
    drop_cols = [c for c in (set(not_use_col) | {target_col}) if c in work.columns]
    X_raw = work.drop(columns=drop_cols, errors="ignore")
    
    for c in X_raw.columns:
        X_raw[c] = pd.to_numeric(X_raw[c], errors="coerce")

    if model_type == "regression":
        y = pd.to_numeric(work[target_col], errors="coerce")
    else:
        y = work[target_col].astype("int64")

    valid_idx = (~X_raw.isnull().all(axis=1)) & (~pd.isnull(y))
    X_raw = X_raw[valid_idx].reset_index(drop=True)
    y = y[valid_idx].reset_index(drop=True)
    dates = dates[valid_idx].reset_index(drop=True)
    
    if split_method == 'stratified':
        if model_type == "classification":
            sss = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=42)
            train_idx, test_idx = next(sss.split(X_raw, y))
        else:
            train_idx, test_idx = train_test_split(
                range(len(X_raw)), test_size=test_size, random_state=42
            )
            
        X_train, X_test = X_raw.iloc[train_idx].copy(), X_raw.iloc[test_idx].copy()
        y_train, y_test = y.iloc[train_idx].copy(), y.iloc[test_idx].copy()
        dates_train, dates_test = dates.iloc[train_idx].copy(), dates.iloc[test_idx].copy()
        
    else:  # temporal split
        n = len(X_raw)
        split = int(n * (1 - test_size))
        X_train, X_test = X_raw.iloc[:split].copy(), X_raw.iloc[split:].copy()
        y_train, y_test = y.iloc[:split].copy(), y.iloc[split:].copy()
        dates_train, dates_test = dates.iloc[:split].copy(), dates.iloc[split:].copy()

    feature_names = list(X_raw.columns)
    return X_train, X_test, y_train, y_test, feature_names, dates_train, dates_test

def make_features(df, cutoff_date=None):
    """파생변수 생성 함수 - Data Leakage 방지 버전"""
    df = df.copy()
    
    df['날짜'] = pd.to_datetime(df['날짜'])
    df = df.sort_values('날짜').reset_index(drop=True)
    
    # 디버깅 정보 - 시작 시점
    print(f"\n=== make_features 디버깅 ===")
    print(f"입력 데이터: {len(df)}행")
    print(f"날짜 범위: {df['날짜'].min()} ~ {df['날짜'].max()}")
    if cutoff_date is not None:
        cutoff = pd.to_datetime(cutoff_date)
        print(f"cutoff_date: {cutoff_date}")
        print(f"  cutoff 이전 데이터: {len(df[df['날짜'] <= cutoff])}행")
        print(f"  cutoff 이후 데이터: {len(df[df['날짜'] > cutoff])}행")

    df['월'] = df['날짜'].dt.month
    df['요일'] = df['날짜'].dt.weekday

    season_map = {'봄': 0, '여름': 1, '가을': 2, '겨울': 3}
    discomfort_map = {'쾌적': 0, '약간 불쾌': 1, '불쾌': 2, '매우 불쾌': 3, '극심한 불쾌': 4}
    df['계절'] = df['계절'].map(season_map).astype('Int64')
    df['불쾌지수등급'] = df['불쾌지수등급'].map(discomfort_map).astype('Int64')

    # 강수량 시차 피처
    df['강수량_1일전'] = df['일_일강수량(mm)'].shift(1)
    df['강수량_2일전'] = df['일_일강수량(mm)'].shift(2)
    df['강수량_1일_누적'] = df['일_일강수량(mm)'].rolling(1, min_periods=1).sum()
    df['강수량_2일_누적'] = df['일_일강수량(mm)'].rolling(2, min_periods=1).sum()
    df['강수량_3일_누적'] = df['일_일강수량(mm)'].rolling(3, min_periods=1).sum()
    df['강수량_5일_누적'] = df['일_일강수량(mm)'].rolling(5, min_periods=1).sum()
    df['강수량_7일_누적'] = df['일_일강수량(mm)'].rolling(7, min_periods=1).sum()

    df['일교차'] = df['일_최고기온(°C)'] - df['일_최저기온(°C)']
    df['폭우_여부'] = (df['일_일강수량(mm)'] >= 80).astype(int)
    
    # 체감온도 계산 (간단 버전)
    T = pd.to_numeric(df.get('일_평균기온(°C)', np.nan), errors='coerce')
    V_ms = pd.to_numeric(df.get('일_평균풍속(m/s)', np.nan), errors='coerce')
    RH = pd.to_numeric(df.get('평균습도(%)', np.nan), errors='coerce')
    
    e = (RH/100.0) * 6.105 * np.exp(17.27*T/(237.7 + T))
    df['체감온도(°C)'] = T + 0.33*e - 0.70*V_ms - 4.00
    
    # 분류용 등급 계산
    q = df['합계'].dropna().quantile([0.15, 0.70, 0.90])
    q15, q70, q90 = float(q.loc[0.15]), float(q.loc[0.70]), float(q.loc[0.90])
    print(f"등급 구분 기준: q15={q15:.0f}, q70={q70:.0f}, q90={q90:.0f}")

    def categorize(x):
        if pd.isna(x):
            return np.nan
        if x < q15:
            return 0
        elif x < q70:
            return 1
        elif x < q90:
            return 2
        else:
            return 3

    df['등급'] = df['합계'].apply(categorize)
    
    # 타겟 변수 생성 (Data Leakage 방지)
    if cutoff_date is not None:
        cutoff = pd.to_datetime(cutoff_date)
        print(f"타겟 변수 생성 (cutoff 적용)")
        
        df['합계_1일후'] = np.nan
        df['합계_2일후'] = np.nan
        df['등급_1일후'] = np.nan
        df['등급_2일후'] = np.nan
        
        # 생성된 타겟 개수 추적
        target_1day_count = 0
        target_2day_count = 0
        
        for i in range(len(df)):
            current_date = df.loc[i, '날짜']
            
            if i + 1 < len(df) and current_date <= cutoff:
                next_date = df.loc[i+1, '날짜']
                if next_date <= cutoff:
                    df.loc[i, '합계_1일후'] = df.loc[i+1, '합계']
                    df.loc[i, '등급_1일후'] = df.loc[i+1, '등급']
                    target_1day_count += 1
            
            if i + 2 < len(df) and current_date <= cutoff:
                next2_date = df.loc[i+2, '날짜']
                if next2_date <= cutoff:
                    df.loc[i, '합계_2일후'] = df.loc[i+2, '합계']
                    df.loc[i, '등급_2일후'] = df.loc[i+2, '등급']
                    target_2day_count += 1
                    
        print(f"  1일후 타겟 생성: {target_1day_count}개")
        print(f"  2일후 타겟 생성: {target_2day_count}개")
        
    else:
        print(f"타겟 변수 생성 (shift 사용)")
        df['합계_1일후'] = df['합계'].shift(-1)
        df['합계_2일후'] = df['합계'].shift(-2)
        df['등급_1일후'] = df['등급'].shift(-1).astype('Int64')
        df['등급_2일후'] = df['등급'].shift(-2).astype('Int64')
        print(f"  1일후 타겟 유효값: {df['합계_1일후'].notna().sum()}개")
        print(f"  2일후 타겟 유효값: {df['합계_2일후'].notna().sum()}개")

    df.attrs['cutoffs'] = {"q15": q15, "q70": q70, "q90": q90}
    
    # dropna 전후 비교
    before_dropna = len(df)
    df = df.dropna().reset_index(drop=True)
    after_dropna = len(df)
    print(f"dropna 처리: {before_dropna}행 → {after_dropna}행 ({before_dropna - after_dropna}행 제거)")
    
    # 2025-06-01 필터링
    before_filter = len(df)
    df = df[df["날짜"] < "2025-06-01"]
    after_filter = len(df)
    print(f"날짜 필터링: {before_filter}행 → {after_filter}행 ({before_filter - after_filter}행 제거)")
    
    # 최종 결과
    if '등급_1일후' in df.columns:
        final_grade_dist = df['등급_1일후'].value_counts().sort_index()
        print(f"최종 등급 분포: {dict(final_grade_dist)}")
    
    print(f"최종 출력: {len(df)}행, {len(df.columns)}컬럼")
    print(f"최종 날짜 범위: {df['날짜'].min()} ~ {df['날짜'].max()}")
    print(f"=== make_features 완료 ===\n")
    
    return df

def make_features_for_prediction(historical_df, future_df):
    """새로운 데이터에 대한 파생변수 생성 (과거 데이터 활용)"""
    combined_df = pd.concat([historical_df, future_df], ignore_index=True)
    combined_df['날짜'] = pd.to_datetime(combined_df['날짜'])
    combined_df = combined_df.sort_values('날짜').reset_index(drop=True)
    
    combined_df['월'] = combined_df['날짜'].dt.month
    combined_df['요일'] = combined_df['날짜'].dt.weekday
    
    season_map = {'봄': 0, '여름': 1, '가을': 2, '겨울': 3}
    discomfort_map = {'쾌적': 0, '약간 불쾌': 1, '불쾌': 2, '매우 불쾌': 3, '극심한 불쾌': 4}
    combined_df['계절'] = combined_df['계절'].map(season_map).astype('Int64')
    combined_df['불쾌지수등급'] = combined_df['불쾌지수등급'].map(discomfort_map).astype('Int64')
    
    # 시차 변수들
    combined_df['강수량_1일전'] = combined_df['일_일강수량(mm)'].shift(1)
    combined_df['강수량_2일전'] = combined_df['일_일강수량(mm)'].shift(2)
    combined_df['강수량_1일_누적'] = combined_df['일_일강수량(mm)'].rolling(1, min_periods=1).sum()
    combined_df['강수량_2일_누적'] = combined_df['일_일강수량(mm)'].rolling(2, min_periods=1).sum()
    combined_df['강수량_3일_누적'] = combined_df['일_일강수량(mm)'].rolling(3, min_periods=1).sum()
    combined_df['강수량_5일_누적'] = combined_df['일_일강수량(mm)'].rolling(5, min_periods=1).sum()
    combined_df['강수량_7일_누적'] = combined_df['일_일강수량(mm)'].rolling(7, min_periods=1).sum()
    
    combined_df['일교차'] = combined_df['일_최고기온(°C)'] - combined_df['일_최저기온(°C)']
    combined_df['폭우_여부'] = (combined_df['일_일강수량(mm)'] >= 80).astype(int)
    
    # 체감온도 계산
    T = pd.to_numeric(combined_df.get('일_평균기온(°C)', np.nan), errors='coerce')
    V_ms = pd.to_numeric(combined_df.get('일_평균풍속(m/s)', np.nan), errors='coerce')
    RH = pd.to_numeric(combined_df.get('평균습도(%)', np.nan), errors='coerce')
    
    e = (RH/100.0) * 6.105 * np.exp(17.27*T/(237.7 + T))
    combined_df['체감온도(°C)'] = T + 0.33*e - 0.70*V_ms - 4.00
    
    # 새 데이터 부분만 반환
    historical_len = len(historical_df)
    return combined_df.iloc[historical_len:].reset_index(drop=True)

def make_simple_features(data):
    """간단한 피처 생성 (시차 변수 제외)"""
    df = data.copy()
    df['날짜'] = pd.to_datetime(df['날짜'])
    df = df.sort_values('날짜').reset_index(drop=True)
    
    # 기본 피처들
    df['월'] = df['날짜'].dt.month
    df['요일'] = df['날짜'].dt.weekday
    
    # 계절/불쾌지수 매핑
    season_map = {'봄': 0, '여름': 1, '가을': 2, '겨울': 3}
    discomfort_map = {'쾌적': 0, '약간 불쾌': 1, '불쾌': 2, '매우 불쾌': 3, '극심한 불쾌': 4}
    
    if '계절' in df.columns:
        df['계절'] = df['계절'].map(season_map).astype('Int64')
    if '불쾌지수등급' in df.columns:
        df['불쾌지수등급'] = df['불쾌지수등급'].map(discomfort_map).astype('Int64')
    
    # 기본 계산 피처들
    if '일_최고기온(°C)' in df.columns and '일_최저기온(°C)' in df.columns:
        df['일교차'] = df['일_최고기온(°C)'] - df['일_최저기온(°C)']
    
    if '일_일강수량(mm)' in df.columns:
        df['폭우_여부'] = (df['일_일강수량(mm)'] >= 80).astype(int)
    
    # 체감온도 계산
    T = pd.to_numeric(df.get('일_평균기온(°C)', np.nan), errors='coerce')
    V_ms = pd.to_numeric(df.get('일_평균풍속(m/s)', np.nan), errors='coerce')
    RH = pd.to_numeric(df.get('평균습도(%)', np.nan), errors='coerce')
    
    e = (RH/100.0) * 6.105 * np.exp(17.27*T/(237.7 + T))
    df['체감온도(°C)'] = T + 0.33*e - 0.70*V_ms - 4.00
    
    return df

# ================================================================================================
# 3. 평가 함수들
# ================================================================================================
def evaluate_regression_model(model, model_name, X_train, X_test, y_train, y_test):
    """회귀 모델 평가"""
    try:
        pipe = make_pipeline_unified(model, model_name, "regression")
        pipe.fit(X_train, y_train)
        
        y_pred = pipe.predict(X_test)
        
        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)
        mape = np.mean(np.abs((y_test - y_pred) / (y_test + 1e-8))) * 100
        
        return {
            'model': model_name,
            'type': 'regression',
            'mae': mae,
            'rmse': rmse,
            'r2': r2,
            'mape': mape,
            'success': True
        }, pipe, y_pred
        
    except Exception as e:
        return {
            'model': model_name,
            'type': 'regression',
            'mae': np.nan,
            'rmse': np.nan,
            'r2': np.nan,
            'mape': np.nan,
            'success': False,
            'error': str(e)
        }, None, None

def evaluate_classification_model(model, model_name, X_train, X_test, y_train, y_test):
    """분류 모델 평가"""
    try:
        pipe = make_pipeline_unified(model, model_name, "classification")
        pipe.fit(X_train, y_train)
        
        y_pred = pipe.predict(X_test)
        
        if isinstance(y_pred, np.ndarray) and y_pred.ndim > 1:
            y_pred = y_pred.ravel()
        
        acc = accuracy_score(y_test, y_pred)
        f1_macro = f1_score(y_test, y_pred, average="macro", zero_division=0)
        f1_weighted = f1_score(y_test, y_pred, average="weighted", zero_division=0)
        
        extreme_classes = [0, 3]
        y_true_extreme = pd.Series(y_test).isin(extreme_classes).astype(int)
        y_pred_extreme = pd.Series(y_pred).isin(extreme_classes).astype(int)
        extreme_f1 = f1_score(y_true_extreme, y_pred_extreme, zero_division=0)
        
        return {
            'model': model_name,
            'type': 'classification',
            'accuracy': acc,
            'macro_f1': f1_macro,
            'weighted_f1': f1_weighted,
            'extreme_f1': extreme_f1,
            'success': True
        }, pipe, y_pred
        
    except Exception as e:
        return {
            'model': model_name,
            'type': 'classification',
            'accuracy': np.nan,
            'macro_f1': np.nan,
            'weighted_f1': np.nan,
            'extreme_f1': np.nan,
            'success': False,
            'error': str(e)
        }, None, None

def comprehensive_evaluation_comparison(center_name, df):
    """Stratified vs 시계열 분할 비교 평가"""
    print(f"\n{'='*70}")
    print(f"센터: {center_name} - Stratified vs 시계열 분할 비교")
    print(f"{'='*70}")
    
    print(f"데이터 크기: {len(df)}행, {len(df.columns)}컬럼")
    
    if '등급_1일후' in df.columns:
        grade_dist = df['등급_1일후'].value_counts().sort_index()
        print(f"등급 분포: {dict(grade_dist)}")
        
        min_class = grade_dist.min()
        max_class = grade_dist.max()
        imbalance_ratio = max_class / min_class
        print(f"클래스 불균형 비율: {imbalance_ratio:.1f}:1 (최대:{max_class}, 최소:{min_class})")
    
    results = []
    
    for split_method in ['temporal', 'stratified']:
        print(f"\n{'='*50}")
        print(f"분할 방법: {split_method.upper()}")
        print(f"{'='*50}")
        
        # 회귀 모델 평가
        reg_method_name = "random_shuffle" if split_method == "stratified" else split_method
        print(f"\n--- 회귀 모델 평가 ({reg_method_name}) ---")
        
        try:
            X_train, X_test, y_train, y_test, feature_names, dates_train, dates_test = prepare_data_stratified(
                df, target_col="합계_1일후", model_type="regression", test_size=0.2, split_method=split_method
            )
            
            print(f"회귀용 데이터: 학습 {len(X_train)}행, 테스트 {len(X_test)}행")
            
            regression_models = build_regression_models()
            
            for model_name, model in tqdm(regression_models.items(), desc=f"회귀({reg_method_name})", leave=False):
                result, pipe, y_pred = evaluate_regression_model(model, model_name, X_train, X_test, y_train, y_test)
                result['center'] = center_name
                result['split_method'] = split_method
                results.append(result)
                
                if result['success']:
                    print(f"  {model_name:18s}: R²={result['r2']:.3f}, MAE={result['mae']:.0f}, MAPE={result['mape']:.1f}%")
                else:
                    print(f"  {model_name:18s}: 실패 - {result.get('error', '')[:50]}")
                    
        except Exception as e:
            print(f"회귀 모델 평가 실패 ({reg_method_name}): {e}")
        
        # 분류 모델 평가
        print(f"\n--- 분류 모델 평가 ({split_method}) ---")
        
        try:
            X_train_clf, X_test_clf, y_train_clf, y_test_clf, feature_names_clf, _, _ = prepare_data_stratified(
                df, target_col="등급_1일후", model_type="classification", test_size=0.2, split_method=split_method
            )
            
            print(f"분류용 데이터: 학습 {len(X_train_clf)}행, 테스트 {len(X_test_clf)}행")
            
            test_dist = pd.Series(y_test_clf).value_counts().sort_index()
            train_dist = pd.Series(y_train_clf).value_counts().sort_index()
            print(f"학습 세트 등급 분포: {dict(train_dist)}")
            print(f"테스트 세트 등급 분포: {dict(test_dist)}")
            
            classification_models = build_classification_models()
            
            for model_name, model in tqdm(classification_models.items(), desc=f"분류({split_method})", leave=False):
                result, pipe, y_pred = evaluate_classification_model(model, model_name, X_train_clf, X_test_clf, y_train_clf, y_test_clf)
                result['center'] = center_name
                result['split_method'] = split_method
                results.append(result)
                
                if result['success']:
                    print(f"  {model_name:18s}: ACC={result['accuracy']:.3f}, F1={result['macro_f1']:.3f}, 극값F1={result['extreme_f1']:.3f}")
                else:
                    print(f"  {model_name:18s}: 실패 - {result.get('error', '')[:50]}")
                    
        except Exception as e:
            print(f"분류 모델 평가 실패 ({split_method}): {e}")
    
    return results

# ================================================================================================
# 4. Feature Importance & SHAP 분석 함수들 (선택사항)
# ================================================================================================
def extract_feature_importance(model, model_name, feature_names):
    """모델별 Feature Importance 추출"""
    try:
        mdl = model.named_steps['model']
        if hasattr(mdl, 'feature_importances_'):
            importance = mdl.feature_importances_
        elif hasattr(mdl, 'coef_'):
            coef = mdl.coef_
            if isinstance(coef, np.ndarray) and coef.ndim == 2:
                importance = np.mean(np.abs(coef), axis=0)
            else:
                importance = np.abs(coef)
        else:
            return None

        if len(importance) != len(feature_names):
            print(f"[경고] importance 길이({len(importance)}) != feature_names({len(feature_names)})")
            m = min(len(importance), len(feature_names))
            importance = np.asarray(importance)[:m]
            feature_names = list(feature_names)[:m]

        importance_df = pd.DataFrame({
            'feature': feature_names,
            'importance': importance
        }).sort_values('importance', ascending=False)

        return importance_df
    except Exception as e:
        print(f"Feature importance 추출 실패 ({model_name}): {e}")
        return None

def plot_feature_importance(importance_df, model_name, top_n=15):
    """Feature Importance 시각화"""
    if importance_df is None or len(importance_df) == 0:
        return None
    
    fig, ax = plt.subplots(figsize=(10, 8))
    top_features = importance_df.head(top_n)
    
    ax.barh(range(len(top_features)), top_features['importance'], color='skyblue')
    ax.set_yticks(range(len(top_features)))
    ax.set_yticklabels(top_features['feature'])
    ax.set_xlabel('Importance')
    ax.set_title(f'{model_name} - Top {top_n} Feature Importance')
    ax.invert_yaxis()
    
    for i, v in enumerate(top_features['importance']):
        ax.text(v + 0.001, i, f'{v:.3f}', va='center')
    
    fig.tight_layout()
    return fig

def analyze_model_with_shap(model, X_test, feature_names, model_name, max_samples=100):
    """SHAP 분석 - 다중분류 대응 강화"""
    if not HAS_SHAP:
        print("SHAP 라이브러리가 설치되지 않았습니다.")
        return None
    
    try:
        if len(X_test) > max_samples:
            sample_idx = np.random.choice(len(X_test), max_samples, replace=False)
            X_sample = X_test.iloc[sample_idx]
        else:
            X_sample = X_test
        
        X_processed = model.named_steps['pre'].transform(X_sample)
        
        if 'RandomForest' in model_name or 'GradientBoosting' in model_name:
            explainer = shap.TreeExplainer(model.named_steps['model'])
        elif 'XGBoost' in model_name:
            explainer = shap.TreeExplainer(model.named_steps['model'])
        elif 'LightGBM' in model_name:
            explainer = shap.TreeExplainer(model.named_steps['model'])
        elif 'CatBoost' in model_name:
            explainer = shap.TreeExplainer(model.named_steps['model'])
        else:
            explainer = shap.LinearExplainer(model.named_steps['model'], X_processed)
        
        shap_values = explainer.shap_values(X_processed)
        
        # 원본 SHAP 값 디버깅
        print(f"원본 SHAP 값 타입: {type(shap_values)}")
        if isinstance(shap_values, list):
            print(f"리스트 길이: {len(shap_values)}")
            if len(shap_values) > 0:
                print(f"첫 번째 요소 shape: {shap_values[0].shape if hasattr(shap_values[0], 'shape') else 'no shape'}")
        elif hasattr(shap_values, 'shape'):
            print(f"배열 shape: {shap_values.shape}")
        
        return shap_values, X_processed, explainer
        
    except Exception as e:
        print(f"SHAP 분석 실패 ({model_name}): {e}")
        return None

def plot_shap_summary(shap_values, X_processed, feature_names, model_name):
    """SHAP Summary Plot - 완전 수정 버전"""
    if shap_values is None or not HAS_SHAP:
        return []

    figs = []
    try:
        # 다중분류 SHAP 값 처리 - 더 세밀한 처리
        shap_values_use = None
        
        if isinstance(shap_values, list):
            print(f"SHAP 값이 리스트, 길이: {len(shap_values)}")
            if len(shap_values) > 0:
                # 다중분류의 경우 첫 번째 클래스만 사용
                shap_values_use = shap_values[0]
                print(f"첫 번째 클래스 shape: {shap_values_use.shape}")
        elif isinstance(shap_values, np.ndarray):
            if shap_values.ndim == 3:
                print(f"3차원 배열: {shap_values.shape}")
                # (samples, features, classes) -> (samples, features)
                shap_values_use = shap_values[:, :, 0]
                print(f"첫 번째 클래스 추출 후: {shap_values_use.shape}")
            elif shap_values.ndim == 2:
                print(f"2차원 배열: {shap_values.shape}")
                shap_values_use = shap_values
            else:
                print(f"예상치 못한 차원: {shap_values.ndim}")
                shap_values_use = shap_values
        else:
            print(f"알 수 없는 타입: {type(shap_values)}")
            shap_values_use = shap_values
        
        # 최종 확인
        if shap_values_use is None:
            print("SHAP 값 처리 실패")
            return []
        
        print(f"최종 사용할 SHAP 값 shape: {shap_values_use.shape}")
        print(f"X_processed shape: {X_processed.shape}")
        print(f"feature_names 길이: {len(feature_names)}")
        
        # 차원 일치성 확인
        if hasattr(shap_values_use, 'shape') and len(shap_values_use.shape) >= 2:
            if shap_values_use.shape[1] != len(feature_names):
                print(f"경고: SHAP 피처 수({shap_values_use.shape[1]}) != feature_names 수({len(feature_names)})")
                # 최소 길이로 맞춤
                min_features = min(shap_values_use.shape[1], len(feature_names))
                shap_values_use = shap_values_use[:, :min_features]
                feature_names = feature_names[:min_features]
                print(f"조정 후 - SHAP: {shap_values_use.shape}, features: {len(feature_names)}")
        
        # Bar plot 시도
        try:
            plt.figure(figsize=(10, 8))
            shap.summary_plot(shap_values_use, X_processed[:len(shap_values_use)],
                              feature_names=feature_names,
                              plot_type="bar", show=False)
            plt.title(f'{model_name} - SHAP Feature Importance')
            figs.append(plt.gcf())
            print("    SHAP bar plot 성공")
        except Exception as e:
            print(f"    SHAP bar plot 실패: {e}")
        
        # Beeswarm plot 시도 (더 까다로움)
        try:
            plt.figure(figsize=(10, 8))
            shap.summary_plot(shap_values_use, X_processed[:len(shap_values_use)],
                              feature_names=feature_names,
                              show=False)
            plt.title(f'{model_name} - SHAP Summary Plot')
            figs.append(plt.gcf())
            print("    SHAP beeswarm plot 성공")
        except Exception as e:
            print(f"    SHAP beeswarm plot 실패: {e}")

    except Exception as e:
        print(f"SHAP 시각화 전체 실패 ({model_name}): {e}")

    plt.close('all')
    return figs

# ================================================================================================
# 5. 유틸리티 함수들
# ================================================================================================
def load_original_data():
    """원본 데이터 로드"""
    nanji_raw = pd.read_csv('../data/processed/center_season/nanji/난지_merged.csv', encoding='utf-8-sig')
    jungnang_raw = pd.read_csv('../data/processed/center_season/jungnang/중랑_merged.csv', encoding='utf-8-sig')
    seonam_raw = pd.read_csv('../data/processed/center_season/seonam/서남_merged.csv', encoding='utf-8-sig')
    tancheon_raw = pd.read_csv('../data/processed/center_season/tancheon/탄천_merged.csv', encoding='utf-8-sig')
    
    return {
        "nanji": nanji_raw,
        "jungnang": jungnang_raw,
        "seonam": seonam_raw,
        "tancheon": tancheon_raw
    }

def prepare_prediction_features(future_data, expected_features):
    """예측용 피처 준비"""
    not_use_col = [
        '날짜', '1처리장','2처리장','정화조','중계펌프장','합계','시설현대화',
        '3처리장','4처리장','합계', '합계_1일후','합계_2일후',
        '등급','등급_1일후','등급_2일후'
    ]
    
    available_cols = [col for col in future_data.columns if col not in not_use_col]
    X_future = future_data[available_cols].copy()
    
    for c in X_future.columns:
        X_future[c] = pd.to_numeric(X_future[c], errors="coerce")
    
    missing_features = set(expected_features) - set(X_future.columns)
    if missing_features:
        for feature in missing_features:
            X_future[feature] = 0
    
    X_future = X_future[expected_features].copy()
    return X_future

# ================================================================================================
# 6. 모델 성능 비교 시각화 함수들
# ================================================================================================
def plot_data_characteristics_comparison(centers_data, results_dir, timestamp):
    """센터별 데이터 특성 비교"""
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('센터별 데이터 특성 비교', fontsize=16, fontweight='bold')
    
    center_stats = []
    
    for center_name, df in centers_data.items():
        df['날짜'] = pd.to_datetime(df['날짜'])
        df = df.sort_values('날짜').reset_index(drop=True)
        
        stats = {
            'center': center_name,
            'total_records': len(df),
            'date_range': (df['날짜'].max() - df['날짜'].min()).days,
            'avg_flow': df['합계'].mean() if '합계' in df.columns else 0,
            'std_flow': df['합계'].std() if '합계' in df.columns else 0,
            'missing_ratio': df.isnull().sum().sum() / (len(df) * len(df.columns))
        }
        center_stats.append(stats)
    
    center_stats_df = pd.DataFrame(center_stats)
    
    # 1. 데이터 양 비교
    axes[0,0].bar(center_stats_df['center'], center_stats_df['total_records'], color='skyblue')
    axes[0,0].set_title('센터별 데이터 양')
    axes[0,0].set_ylabel('레코드 수')
    axes[0,0].tick_params(axis='x', rotation=45)
    for i, v in enumerate(center_stats_df['total_records']):
        axes[0,0].text(i, v + 10, str(v), ha='center', va='bottom')
    
    # 2. 평균 유량 비교
    axes[0,1].bar(center_stats_df['center'], center_stats_df['avg_flow'], color='lightgreen')
    axes[0,1].set_title('센터별 평균 유량')
    axes[0,1].set_ylabel('평균 유량')
    axes[0,1].tick_params(axis='x', rotation=45)
    for i, v in enumerate(center_stats_df['avg_flow']):
        axes[0,1].text(i, v + max(center_stats_df['avg_flow'])*0.01, f'{v:.0f}', ha='center', va='bottom')
    
    # 3. 유량 변동성 비교
    axes[1,0].bar(center_stats_df['center'], center_stats_df['std_flow'], color='coral')
    axes[1,0].set_title('센터별 유량 변동성 (표준편차)')
    axes[1,0].set_ylabel('표준편차')
    axes[1,0].tick_params(axis='x', rotation=45)
    for i, v in enumerate(center_stats_df['std_flow']):
        axes[1,0].text(i, v + max(center_stats_df['std_flow'])*0.01, f'{v:.0f}', ha='center', va='bottom')
    
    # 4. 데이터 완성도
    missing_pct = center_stats_df['missing_ratio'] * 100
    axes[1,1].bar(center_stats_df['center'], 100 - missing_pct, color='gold')
    axes[1,1].set_title('센터별 데이터 완성도')
    axes[1,1].set_ylabel('완성도 (%)')
    axes[1,1].set_ylim(0, 100)
    axes[1,1].tick_params(axis='x', rotation=45)
    for i, v in enumerate(100 - missing_pct):
        axes[1,1].text(i, v + 1, f'{v:.1f}%', ha='center', va='bottom')
    
    plt.tight_layout()
    save_path = os.path.join(results_dir, f"data_characteristics_comparison_{timestamp}.png")
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"  데이터 특성 비교 저장: {os.path.basename(save_path)}")

def plot_model_complexity_vs_performance(training_summary, results_dir, timestamp):
    """모델 복잡도 vs 성능 관계"""
    complexity_map = {
        'LinearRegression': 1, 'LogisticRegression_Clf': 1,
        'RandomForest_Reg': 3, 'RandomForest_Clf': 3,
        'GradientBoosting_Reg': 4, 'GradientBoosting_Clf': 4,
        'XGBoost_Reg': 4, 'XGBoost_Clf': 4,
        'LightGBM_Reg': 4, 'LightGBM_Clf': 4,
        'CatBoost_Reg': 5, 'CatBoost_Clf': 5
    }
    
    if isinstance(training_summary, list):
        summary_df = pd.DataFrame(training_summary)
    else:
        summary_df = training_summary.copy()
    
    summary_df['complexity'] = summary_df['model_name'].map(complexity_map)
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    fig.suptitle('모델 복잡도 vs 성능 관계', fontsize=16, fontweight='bold')
    
    # 회귀 모델
    reg_data = summary_df[summary_df['task_type'] == 'regression'].copy()
    if len(reg_data) > 0:
        reg_data['r2'] = reg_data['performance'].apply(lambda x: x.get('r2', 0))
        
        scatter = axes[0].scatter(reg_data['complexity'], reg_data['r2'], 
                                 c=reg_data.index, cmap='viridis', s=100, alpha=0.7)
        
        for i, row in reg_data.iterrows():
            axes[0].annotate(f"{row['center'][:3]}", 
                           (row['complexity'], row['r2']),
                           xytext=(5, 5), textcoords='offset points', fontsize=8)
        
        axes[0].set_xlabel('모델 복잡도 (1=단순, 5=복잡)')
        axes[0].set_ylabel('R² Score')
        axes[0].set_title('회귀: 복잡도 vs 성능')
        axes[0].grid(True, alpha=0.3)
        axes[0].set_xticks(range(1, 6))
    
    # 분류 모델
    clf_data = summary_df[summary_df['task_type'] == 'classification'].copy()
    if len(clf_data) > 0:
        clf_data['macro_f1'] = clf_data['performance'].apply(lambda x: x.get('macro_f1', 0))
        
        scatter = axes[1].scatter(clf_data['complexity'], clf_data['macro_f1'], 
                                 c=clf_data.index, cmap='plasma', s=100, alpha=0.7)
        
        for i, row in clf_data.iterrows():
            axes[1].annotate(f"{row['center'][:3]}", 
                           (row['complexity'], row['macro_f1']),
                           xytext=(5, 5), textcoords='offset points', fontsize=8)
        
        axes[1].set_xlabel('모델 복잡도 (1=단순, 5=복잡)')
        axes[1].set_ylabel('Macro F1 Score')
        axes[1].set_title('분류: 복잡도 vs 성능')
        axes[1].grid(True, alpha=0.3)
        axes[1].set_xticks(range(1, 6))
    
    plt.tight_layout()
    save_path = os.path.join(results_dir, f"model_complexity_vs_performance_{timestamp}.png")
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"  복잡도 vs 성능 저장: {os.path.basename(save_path)}")

def plot_prediction_difficulty_analysis(training_summary, results_dir, timestamp):
    """센터별 예측 난이도 분석"""
    if isinstance(training_summary, list):
        summary_df = pd.DataFrame(training_summary)
    else:
        summary_df = training_summary.copy()
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('센터별 예측 난이도 분석', fontsize=16, fontweight='bold')
    
    centers = summary_df['center'].unique()
    
    # 1. 센터별 최고 R² 성능
    reg_best_scores = []
    for center in centers:
        center_reg = summary_df[(summary_df['center'] == center) & 
                               (summary_df['task_type'] == 'regression')]
        if len(center_reg) > 0:
            center_reg = center_reg.copy()
            center_reg['r2'] = center_reg['performance'].apply(lambda x: x.get('r2', 0))
            reg_best_scores.append(center_reg['r2'].max())
        else:
            reg_best_scores.append(0)
    
    bars1 = axes[0,0].bar(centers, reg_best_scores, color='skyblue')
    axes[0,0].set_title('센터별 최고 R² 성능 (높을수록 예측 용이)')
    axes[0,0].set_ylabel('최고 R² Score')
    axes[0,0].set_ylim(0, 1)
    axes[0,0].tick_params(axis='x', rotation=45)
    
    for bar, score in zip(bars1, reg_best_scores):
        axes[0,0].text(bar.get_x() + bar.get_width()/2, score + 0.01, 
                      f'{score:.3f}', ha='center', va='bottom')
    
    # 2. 센터별 성능 분산 (일관성)
    reg_score_vars = []
    for center in centers:
        center_reg = summary_df[(summary_df['center'] == center) & 
                               (summary_df['task_type'] == 'regression')]
        if len(center_reg) > 0:
            center_reg = center_reg.copy()
            center_reg['r2'] = center_reg['performance'].apply(lambda x: x.get('r2', 0))
            reg_score_vars.append(center_reg['r2'].std())
        else:
            reg_score_vars.append(0)
    
    bars2 = axes[0,1].bar(centers, reg_score_vars, color='lightcoral')
    axes[0,1].set_title('센터별 성능 분산 (낮을수록 일관성 높음)')
    axes[0,1].set_ylabel('R² Score 표준편차')
    axes[0,1].tick_params(axis='x', rotation=45)
    
    for bar, var in zip(bars2, reg_score_vars):
        axes[0,1].text(bar.get_x() + bar.get_width()/2, var + max(reg_score_vars)*0.01, 
                      f'{var:.3f}', ha='center', va='bottom')
    
    # 3. 센터별 최고 F1 성능
    clf_best_scores = []
    for center in centers:
        center_clf = summary_df[(summary_df['center'] == center) & 
                               (summary_df['task_type'] == 'classification')]
        if len(center_clf) > 0:
            center_clf = center_clf.copy()
            center_clf['macro_f1'] = center_clf['performance'].apply(lambda x: x.get('macro_f1', 0))
            clf_best_scores.append(center_clf['macro_f1'].max())
        else:
            clf_best_scores.append(0)
    
    bars3 = axes[1,0].bar(centers, clf_best_scores, color='lightgreen')
    axes[1,0].set_title('센터별 최고 F1 성능 (높을수록 분류 용이)')
    axes[1,0].set_ylabel('최고 Macro F1 Score')
    axes[1,0].set_ylim(0, 1)
    axes[1,0].tick_params(axis='x', rotation=45)
    
    for bar, score in zip(bars3, clf_best_scores):
        axes[1,0].text(bar.get_x() + bar.get_width()/2, score + 0.01, 
                      f'{score:.3f}', ha='center', va='bottom')
    
    # 4. 종합 예측 난이도 점수
    difficulty_scores = [(r2 + f1) / 2 for r2, f1 in zip(reg_best_scores, clf_best_scores)]
    colors = plt.cm.RdYlGn([score for score in difficulty_scores])
    
    bars4 = axes[1,1].bar(centers, difficulty_scores, color=colors)
    axes[1,1].set_title('종합 예측 용이도 점수')
    axes[1,1].set_ylabel('종합 점수 (R² + F1) / 2')
    axes[1,1].set_ylim(0, 1)
    axes[1,1].tick_params(axis='x', rotation=45)
    
    for bar, score in zip(bars4, difficulty_scores):
        axes[1,1].text(bar.get_x() + bar.get_width()/2, score + 0.01, 
                      f'{score:.3f}', ha='center', va='bottom')
    
    plt.tight_layout()
    save_path = os.path.join(results_dir, f"prediction_difficulty_analysis_{timestamp}.png")
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"  예측 난이도 분석 저장: {os.path.basename(save_path)}")

    """Split Method 상세 비교 분석 - 수정된 버전"""
    if isinstance(all_training_results, list):
        summary_df = pd.DataFrame(all_training_results)
    else:
        summary_df = all_training_results.copy()
    
    successful_results = summary_df[summary_df['success'] == True].copy()
    
    if len(successful_results) == 0 or 'split_method' not in successful_results.columns:
        print("Split method 정보가 없거나 성공한 결과가 없습니다.")
        return
    
    fig, axes = plt.subplots(2, 3, figsize=(20, 12))
    fig.suptitle('Stratified vs Temporal Split 상세 비교', fontsize=16, fontweight='bold')
    
    # 회귀 데이터 처리
    reg_data = successful_results[successful_results['type'] == 'regression'].copy()
    if len(reg_data) > 0:
        reg_pivot = reg_data.pivot_table(index='center', columns='split_method', values='r2')
        reg_pivot.plot(kind='bar', ax=axes[0,0])
        axes[0,0].set_title('센터별 R² 성능: Split Method 비교')
        axes[0,0].set_ylabel('R² Score')
        axes[0,0].tick_params(axis='x', rotation=45)
        axes[0,0].legend(title='Split Method')
        
        split_methods = reg_data['split_method'].unique()
        r2_by_split = [reg_data[reg_data['split_method'] == method]['r2'].values 
                       for method in split_methods]
        
        axes[0,1].boxplot(r2_by_split, labels=split_methods)
        axes[0,1].set_title('Split Method별 R² 분포')
        axes[0,1].set_ylabel('R² Score')
        axes[0,1].grid(True, alpha=0.3)
        
        if len(split_methods) == 2:
            method1, method2 = split_methods
            perf_diff = []
            centers = reg_data['center'].unique()
            
            for center in centers:
                center_data = reg_data[reg_data['center'] == center]
                perf1 = center_data[center_data['split_method'] == method1]['r2']
                perf2 = center_data[center_data['split
                
                

In [ ]:
saved_models, summary = train_and_save_best_models_with_plots_and_shap()